In [ ]:
"""
ai.py  – High‑level orchestration layer for the Ask Aglio chatbot.

`generate_blocks(question, session_id)`:
    • Maintains OpenAI conversational context (storing the last response_id in Redis)
    • Lets the model call our backend “tools” (function‑calling)
    • Converts the result into FE‑ready `blocks` (validated)
"""
from loguru import logger
import os, json
from typing import List, Dict, Any
from dotenv import load_dotenv

from openai import OpenAI
from redis import Redis

from config import rdb  # redis instance from your config
from recommender.tools_registry import openai_tools
from recommender import SYSTEM_PROMPT
from recommender import tools

load_dotenv()
client = OpenAI()                # assumes OPENAI_API_KEY env var


# ------------------------------------------------------------ #
#  Helpers
# ------------------------------------------------------------ #
_TOOL_MAP = {
    "search_menu": tools.search_menu,
    "get_chefs_picks": tools.get_chefs_picks,
    "list_all_items": tools.list_all_items,
    "find_similar_items": tools.find_similar_items,
    "budget_friendly_options": tools.budget_friendly_options,
    # "describe_dish": tools.describe_dish,
    "get_cart_pairings": tools.get_cart_pairings,
    # "validate_blocks": tools.validate_blocks,
}

In [ ]:
from pydantic import BaseModel
from typing import List, Dict, Any, Literal

class DishItem(BaseModel):
    id: int
    name: str

class TextBlock(BaseModel):
    type: Literal["text"]
    markdown: str

class DishCarouselBlock(BaseModel):
    type: Literal["dish_carousal"]
    options: List[DishItem]

class QuickRepliesBlock(BaseModel):
    type: Literal["quick_replies"]
    options: List[str]

class Blocks(BaseModel):
    blocks: List[TextBlock | DishCarouselBlock | QuickRepliesBlock | DishItem]
    

In [ ]:
prev_id = None
question = "Chefs Specials?"
filters = {
    "veg": True,
    "category_brief": ["Pasta", "Pizza"],
}
cart = []

msgs = []
if not prev_id:
    system_msg = {
        "role": "system",
        "content": SYSTEM_PROMPT,
    }
    msgs.append(system_msg)
        
msg = {
    "role": "user",
    "content": question,
}
msgs.append(msg)

In [ ]:
# 1️⃣  Send user message to OpenAI with tool schemas
counter = iter(range(0, 5))
responses = []

while True:
    print(next(counter))
    response = client.responses.parse(
        model="gpt-4.1",
        input=msgs,
        previous_response_id=prev_id,
        tools=openai_tools,
        tool_choice="auto",
        text_format=Blocks
    )
    msgs = []
    # Persist context id for next turn
    prev_id = response.id

    print(response.model_dump())

    check_tools_calls = any([output.type == "function_call" for output in response.output])

    # 2️⃣  If the model wants to call a function
    if check_tools_calls:
        for tool_call in response.output:
            fn_name   = tool_call.name
            fn_args   = json.loads(tool_call.arguments)
            call_id = tool_call.call_id

            if fn_name not in _TOOL_MAP:
                logger.error("Unknown tool call requested: %s", fn_name)
                raise Exception("Unknown tool call requested: %s" % fn_name)

            # Call the local Python helper
            if fn_name in ["search_menu", "get_chefs_picks", "list_all_items", "find_similar_items", "budget_friendly_options", "get_cart_pairings"]:
                fn_args["filters"] = filters

            if fn_name == "get_cart_pairings":
                fn_args["cart"] = cart

            result = _TOOL_MAP[fn_name](**fn_args)

            msgs.append({"type": "function_call_output", "output": str(result), "call_id": call_id})
        continue
    else:
        # WRITE EXIT ROUTINE AND ENRICHMENT
        blocks = response.output_parsed
        break

blocks


In [ ]:
blocks.model_dump()